In [ ]:
import numpy as np
import scipy as sp
import scipy.linalg
import cvxpy as cp
import random

In [ ]:
def NKron(*args):
  result = np.array([[1.0]])
  for op in args:
    result = np.kron(result, op)
  return result

In [ ]:
Id = np.eye(2)
X = np.array([[0.0, 1.0],[1.0, 0.0]])
Z = np.array([[1.0, 0.0],[0.0, -1.0]])
Y = np.matmul(X,Z)

In [ ]:
NormalizeState = lambda state: state / sp.linalg.norm(state)
zero = np.array([[1.0], [0.0]]) # |0>
one = np.array([[0.0], [1.0]]) # |1>

## Generators

In [ ]:
def NKronModified(checkRowMod):
  result = np.array([[1.0]])
  for ind in checkRowMod:
    if(ind == 0):
        op = Id
    elif(ind == 1):
        op = X
    elif(ind == 2):
        op = Y
    elif(ind == 3):
        op = Z
    result = np.kron(result, op)
  return result

def getGenerator(checkRow):
    checkRowModified = np.zeros(n, dtype=int)
    
    checkRowModified[(checkRow[:n] == checkRow[n:]) & (checkRow[n:] == 1)] = 2
    checkRowModified[(checkRow[:n] == 1) & (checkRowModified != 2)] = 1
    checkRowModified[(checkRow[n:] == 1) & (checkRowModified != 2)] = 3
    
    return NKronModified(checkRowModified)    

In [ ]:
# change check matrix here

# checkMatrix = np.array([[0,0,0,0,0,0,0,0,0, 1,1,0,0,0,0,0,0,0],
#                         [0,0,0,0,0,0,0,0,0, 1,0,1,0,0,0,0,0,0],
#                         [0,0,0,0,0,0,0,0,0, 0,0,0,1,1,0,0,0,0],
#                         [0,0,0,0,0,0,0,0,0, 0,0,0,1,0,1,0,0,0],
#                         [0,0,0,0,0,0,0,0,0, 0,0,0,0,0,0,1,1,0],
#                         [0,0,0,0,0,0,0,0,0, 0,0,0,0,0,0,1,0,1],
#                         [1,1,1,1,1,1,0,0,0, 0,0,0,0,0,0,0,0,0],
#                         [1,1,1,0,0,0,1,1,1, 0,0,0,0,0,0,0,0,0]])

# checkMatrix = np.array([[1,0,0,1,0, 0,1,1,0,0],
#                         [0,1,0,0,1, 0,0,1,1,0],
#                         [1,0,1,0,0, 0,0,0,1,1],
#                         [0,1,0,1,0, 1,0,0,0,1]])

checkMatrix = np.array([[0,0,0,1,1,1,1, 0,0,0,0,0,0,0],
                        [0,1,1,0,0,1,1, 0,0,0,0,0,0,0],
                        [1,0,1,0,1,0,1, 0,0,0,0,0,0,0],
                        [0,0,0,0,0,0,0, 0,0,0,1,1,1,1],
                        [0,0,0,0,0,0,0, 0,1,1,0,0,1,1],
                        [0,0,0,0,0,0,0, 1,0,1,0,1,0,1]])

n = int(checkMatrix.shape[1]/2)
k = n-checkMatrix.shape[0]

gi = np.zeros([n-k, 2**n, 2**n])
for i in range(n-k):
    gi[i,:,:] = getGenerator(checkMatrix[i,:])

## Syndromes

In [ ]:
def fillSyndromeTable(checkRowCorrect):
    # get error corresponding to the given vector
    err = getGenerator(checkRowCorrect)

    # get syndrome of that error
    syndrVal = np.zeros(n-k,dtype='int')
    for i in range(n-k):
        syndBool = np.all(     np.abs(np.matmul(np.matmul(err, gi[i,:,:]), err.transpose()) - gi[i,:,:]) < 1e-5     )
        if syndBool == True:
            syndrVal[i] = 1
        else:
            syndrVal[i] = -1

    # convert syndrome to index
    syndrIndex = int(''.join(['1' if x else '0' for x in syndrVal==1]), 2)
    
    # if not already updated, update the syndrome table
    if isFilledTable[syndrIndex] == 0:
        errorRecoveryList[syndrIndex, :, :] = err
        isFilledTable[syndrIndex] = 1 

#### Weight 1 errors:

In [ ]:
isFilledTable = np.zeros(2**(n-k))
errorRecoveryList = np.zeros([2**(n-k), 2**n, 2**n])
string_format = '{:0>' + str(n) + '}'

myIndex = 1
while(myIndex < 2**(n)):
    # generate weight 1 vectors
    checkRow = list(string_format.format("{:b}".format(myIndex)))
    checkRow = list(map(int, checkRow))
    
    # weight 1 error with X, then Y and then Z 
    fillSyndromeTable(np.append(np.asarray(checkRow), np.zeros(n, dtype = 'int')))
    fillSyndromeTable(np.append(np.zeros(n, dtype = 'int'), np.asarray(checkRow)))
    fillSyndromeTable(np.append(np.asarray(checkRow), np.asarray(checkRow)))
        
    myIndex = myIndex*2   

In [ ]:
print(str(np.sum(isFilledTable == 1)) + ' entries filled out of total ' + str(isFilledTable.shape[0]) + ' syndromes')

21 entries filled out of total 64 syndromes


#### Weight 2 errors:

In [ ]:
myIndex1 = 1
while(myIndex1 < 2**(n)):
    # generate weight 1 vectors
    checkRow1 = list(string_format.format("{:b}".format(myIndex1)))
    checkRow1 = np.asarray(list(map(int, checkRow1)))
    
    myIndex2 = myIndex1*2
    while(myIndex2 < 2**n):
        # generate another weight 1 vector
        checkRow2 = list(string_format.format("{:b}".format(myIndex2)))
        checkRow2 = np.asarray(list(map(int, checkRow2)))
        
        #generate weight 2 vector
        checkRow3 = list(string_format.format("{:b}".format(myIndex2)))
        checkRow3 = np.asarray(list(map(int, checkRow3)))
        checkRow3[checkRow1 == 1] = 1
        
        # add weight 2 errors with XX, XY, XZ, ...
        fillSyndromeTable(np.append(checkRow3, np.zeros(n, dtype = 'int')))
        fillSyndromeTable(np.append(checkRow3, checkRow1))
        fillSyndromeTable(np.append(checkRow2, checkRow1))
        
        fillSyndromeTable(np.append(checkRow3, checkRow2))
        fillSyndromeTable(np.append(checkRow3, checkRow3))
        fillSyndromeTable(np.append(checkRow2, checkRow3))
        
        fillSyndromeTable(np.append(checkRow1, checkRow2))
        fillSyndromeTable(np.append(checkRow1, checkRow3))      
        fillSyndromeTable(np.append(np.zeros(n, dtype = 'int'), checkRow3))
               
        myIndex2 = myIndex2*2
        
    myIndex1 = myIndex1*2   

In [ ]:
print(str(np.sum(isFilledTable == 1)) + ' entries filled out of total ' + str(isFilledTable.shape[0]) + ' syndromes')

63 entries filled out of total 64 syndromes


#### Weight 3 errors:

In [ ]:
myIndex1 = 1
tempcount = 0
tempc = 0
while(myIndex1 < 2**(n)):
    # generate weight 1 vectors
    checkRowList = np.zeros([n, 4])
    
    checkRow1 = list(string_format.format("{:b}".format(myIndex1)))
    checkRow1 = np.asarray(list(map(int, checkRow1)))
    
    checkRowCombined1 = list(string_format.format("{:b}".format(myIndex1)))
    checkRowCombined1 = np.asarray(list(map(int, checkRowCombined1)))
    
    myIndex2 = myIndex1*2
    while(myIndex2 < 2**n):
        # generate another weight 1 vector
        checkRow2 = list(string_format.format("{:b}".format(myIndex2)))
        checkRow2 = np.asarray(list(map(int, checkRow2)))
        
        checkRowCombined2 = list(string_format.format("{:b}".format(myIndex2)))
        checkRowCombined2 = np.asarray(list(map(int, checkRowCombined2)))
        
        myIndex3 = myIndex2*2
        while(myIndex3 < 2**n):
            # generate another weight 1 vector
            checkRow3 = list(string_format.format("{:b}".format(myIndex3)))
            checkRow3 = np.asarray(list(map(int, checkRow3)))
            
            # generate weight 2 and 3 vectors
            checkRowCombined3 = list(string_format.format("{:b}".format(myIndex3)))
            checkRowCombined3 = np.asarray(list(map(int, checkRowCombined3)))
            
            checkRowCombined4 = list(string_format.format("{:b}".format(myIndex3)))
            checkRowCombined4 = np.asarray(list(map(int, checkRowCombined4)))
            
            checkRowCombined1[checkRow2 == 1] = 1
            checkRowCombined2[checkRow3 == 1] = 1
            checkRowCombined3[checkRow1 == 1] = 1
            
            checkRowCombined4[checkRow2 == 1] = 1
            checkRowCombined4[checkRow1 == 1] = 1
            
            fillSyndromeTable(np.append(checkRowCombined4, np.zeros(n, dtype = 'int')))
            fillSyndromeTable(np.append(checkRowCombined4, checkRow1))
            fillSyndromeTable(np.append(checkRowCombined4, checkRow2))
            fillSyndromeTable(np.append(checkRowCombined4, checkRow3))
            fillSyndromeTable(np.append(checkRowCombined4, checkRowCombined1))
            fillSyndromeTable(np.append(checkRowCombined4, checkRowCombined2))
            fillSyndromeTable(np.append(checkRowCombined4, checkRowCombined3))
            fillSyndromeTable(np.append(checkRowCombined4, checkRowCombined4))
            fillSyndromeTable(np.append(checkRowCombined3, checkRowCombined4))
            fillSyndromeTable(np.append(checkRowCombined2, checkRowCombined4))
            fillSyndromeTable(np.append(checkRowCombined1, checkRowCombined4))
            fillSyndromeTable(np.append(checkRow3, checkRowCombined4))
            fillSyndromeTable(np.append(checkRow2, checkRowCombined4))
            fillSyndromeTable(np.append(checkRow1, checkRowCombined4))
            fillSyndromeTable(np.append(np.zeros(n, dtype = 'int'), checkRowCombined4))
            
            fillSyndromeTable(np.append(checkRowCombined3, checkRowCombined2))
            fillSyndromeTable(np.append(checkRowCombined3, checkRowCombined1))
            fillSyndromeTable(np.append(checkRowCombined3, checkRow2))
            fillSyndromeTable(np.append(checkRowCombined2, checkRowCombined3))
            fillSyndromeTable(np.append(checkRowCombined1, checkRowCombined3))
            fillSyndromeTable(np.append(checkRow2, checkRowCombined3))
            
            fillSyndromeTable(np.append(checkRowCombined2, checkRowCombined1))
            fillSyndromeTable(np.append(checkRowCombined2, checkRow1))
            fillSyndromeTable(np.append(checkRowCombined1, checkRowCombined2))
            fillSyndromeTable(np.append(checkRow1, checkRowCombined2))
            
            fillSyndromeTable(np.append(checkRowCombined1, checkRow3))
            fillSyndromeTable(np.append(checkRow3, checkRowCombined1))

            if(np.sum(isFilledTable == 1) == isFilledTable.shape[0]):
                break
            myIndex3 = myIndex3*2
        
        if(np.sum(isFilledTable == 1) == isFilledTable.shape[0]):
            break
        myIndex2 = myIndex2*2
    
    if(np.sum(isFilledTable == 1) == isFilledTable.shape[0]):
        break
    myIndex1 = myIndex1*2 

In [ ]:
print(str(np.sum(isFilledTable == 1)) + ' entries filled out of total ' + str(isFilledTable.shape[0]) + ' syndromes')

64 entries filled out of total 64 syndromes


## Encode

In [ ]:
def NKron1DGeneral(ipArray):
    result = np.array([[1.0]])
    for i in ipArray:
        if(i==1):
            op = one
        elif(i==0):
            op = zero
        result = np.kron(result, op)
    return result

### _Generalize this_

In [ ]:
# tx_qbits = NormalizeState(0.5*NKron1DGeneral( np.array([0,0,0,0,1]) ) 
#                           + 0.2*NKron1DGeneral( np.array([0,0,0,0,0]) )) # transmit qbits before encoding

# tx_qbits = NormalizeState(0.5*NKron1DGeneral( np.array([1,1,1,1,1]) ) 
#                           + 0.2*NKron1DGeneral( np.array([0,0,0,0,0]) )) # transmit qbits before encoding

# tx_qbits = NormalizeState(0.5*NKron1DGeneral( np.array([0,0,0,0,0,0,0,0,1]) ) 
#                           + 0.2*NKron1DGeneral( np.array([0,0,0,0,0,0,0,0,0]) )) # transmit qbits before encoding

# tx_qbits = NormalizeState(0.5*NKron1DGeneral( np.array([1,1,1,1,1,1,1,1,1]) ) 
#                           + 0.2*NKron1DGeneral( np.array([0,0,0,0,0,0,0,0,0]) )) # transmit qbits before encoding

tx_qbits = NormalizeState(0.5*NKron1DGeneral( np.array([1,1,1,1,1,1,1]) ) 
                          + 0.2*NKron1DGeneral( np.array([0,0,0,0,0,0,0]) )) # transmit qbits before encoding

A = np.eye(2**n, 2**n) # condition on codes

# Choose this if repetition is done
A[0,0] = 0
A[A.shape[0]-1, A.shape[1]-1] = 0

# Choose this if zero padding is done in beginning
# for i in range(2**k):
#     A[i,i] = 0

tx_encoded = NormalizeState(tx_qbits) # encoded transmit qbits
for i in range(n-k):
    tx_encoded = tx_encoded + np.matmul(gi[i,:,:], tx_encoded)
tx_encoded = NormalizeState(tx_encoded) # encoded transmit qbits

In [ ]:
Gmatrix = np.eye(gi[i,:,:].shape[0], gi[i,:,:].shape[1]) # generator matrix corresponding to this code
for i in range(n-k):
    Gmatrix = Gmatrix + np.matmul(gi[i,:,:], Gmatrix)

## Channel

In [ ]:
# generate channel error randomly
errIndex = random.randint(0,n-1)
errType = random.randint(0,2)

errCheckRow = np.zeros(n, dtype = 'int')
errCheckRow[errIndex] = 1
if(errType == 0):
    errCheckRow = np.append(errCheckRow, errCheckRow) # Y error
elif(errType == 1):
    errCheckRow = np.append(errCheckRow, np.zeros(n, dtype = 'int')) # X error
else:
    errCheckRow = np.append(np.zeros(n, dtype = 'int'), errCheckRow) # Z error

channel_error = getGenerator(errCheckRow) # channel error
rx_erry = np.dot(channel_error, tx_encoded) # received qbits with errors

## Decode

#### Syndrome Check

In [ ]:
syndr = np.zeros([n-k, 1]) # syndrome
for i in range(n-k):
    syndr[i] = np.dot(rx_erry.transpose(), np.dot(gi[i,:,:], rx_erry))
print('Syndrome = ' + str(np.ndarray.astype(syndr.transpose().flatten() , int)))

Syndrome = [ 1  1  1  1 -1  1]


#### Error Correction

In [ ]:
# syndrome lookup table
def SyndromeLookUp(syndr):
    errorSyndromeIndex = int(''.join(['1' if x else '0' for x in np.ndarray.astype(syndr.flatten(), int) == 1]), 2)
    recov = errorRecoveryList[errorSyndromeIndex]
    return recov

In [ ]:
recov = SyndromeLookUp(syndr) # error recovery
rx_encoded = np.matmul(recov.transpose(), rx_erry) # received qbits without error but still encoded

print('Number of qubits in error before error correction = ' + str(np.sum(rx_erry[:,0] - tx_encoded[:,0] > 1e-5)))
print('Number of qubits in error after error correction = ' + str(np.sum(rx_encoded[:,0] - tx_encoded[:,0] > 1e-5)))

Number of qubits in error before error correction = 16
Number of qubits in error after error correction = 0


#### Complete Decode

In [ ]:
# setup optimizer to decode
P = np.matmul(Gmatrix.transpose(), Gmatrix)
q = -np.matmul(rx_encoded.transpose(), Gmatrix).flatten()
x = cp.Variable(rx_encoded.shape[0])

# get qbit that is at closest distance to received encoded qbit
prob = cp.Problem(cp.Minimize((1/2)*cp.quad_form(x, P) + q.T@x), [A@x == np.zeros(x.shape[0])])
prob.solve()
rx_decoded = (NormalizeState(x.value)) # received decoded qbits

print('Number of qubits in error after decoding = ' + str(np.sum(np.abs(rx_decoded - tx_qbits.flatten()) > 1e-5 )))

Number of qubits in error after decoding = 0


## Verification for all single qubit errors

In [ ]:
# verify for all single bit errors
for errType in range(3):
    for errIndex in range(n):

        # generate channel error
        errCheckRow = np.zeros(n, dtype = 'int')
        errCheckRow[errIndex] = 1
        if(errType == 0):
            errCheckRow = np.append(errCheckRow, errCheckRow) # Y error
        elif(errType == 1):
            errCheckRow = np.append(errCheckRow, np.zeros(n, dtype = 'int')) # X error
        else:
            errCheckRow = np.append(np.zeros(n, dtype = 'int'), errCheckRow) # Z error

        # apply channel error
        channel_error = getGenerator(errCheckRow)
        rx_erry = np.dot(channel_error, tx_encoded) # received qbits with error

        # get syndrome
        syndr = np.zeros([n-k, 1]) # syndrome
        for i in range(n-k):
            syndr[i] = np.dot(rx_erry.transpose(), np.dot(gi[i,:,:], rx_erry))

        # error correction
        recov = SyndromeLookUp(syndr) # error recovery
        rx_encoded = np.matmul(recov.transpose(), rx_erry) # received qbits without error but still encoded
        
        # setup optimizer to decode completely
        P = np.matmul(Gmatrix.transpose(), Gmatrix)
        q = -np.matmul(rx_encoded.transpose(), Gmatrix).flatten()
        x = cp.Variable(rx_encoded.shape[0])
        
        # solve for qubits numerically by minimizing distance
        prob = cp.Problem(cp.Minimize((1/2)*cp.quad_form(x, P) + q.T@x), [A@x == np.zeros(x.shape[0])])
        prob.solve()
        rx_decoded = (NormalizeState(x.value)) # received decoded qbits

        # print qubit errors
        print('Error = ' + str(errCheckRow)  + ', # Qbits in error, before decoding = ' + str(np.sum(np.abs(rx_encoded[:,0] - tx_encoded[:,0]) > 1e-5)) + ', after decoding = ' + str(np.sum(np.abs(rx_decoded - tx_qbits.flatten()) > 1e-5 )) )

Error = [1 0 0 0 0 0 0 1 0 0 0 0 0 0], # Qbits in error, before decoding = 0, after decoding = 0
Error = [0 1 0 0 0 0 0 0 1 0 0 0 0 0], # Qbits in error, before decoding = 0, after decoding = 0
Error = [0 0 1 0 0 0 0 0 0 1 0 0 0 0], # Qbits in error, before decoding = 0, after decoding = 0
Error = [0 0 0 1 0 0 0 0 0 0 1 0 0 0], # Qbits in error, before decoding = 0, after decoding = 0
Error = [0 0 0 0 1 0 0 0 0 0 0 1 0 0], # Qbits in error, before decoding = 0, after decoding = 0
Error = [0 0 0 0 0 1 0 0 0 0 0 0 1 0], # Qbits in error, before decoding = 0, after decoding = 0
Error = [0 0 0 0 0 0 1 0 0 0 0 0 0 1], # Qbits in error, before decoding = 0, after decoding = 0
Error = [1 0 0 0 0 0 0 0 0 0 0 0 0 0], # Qbits in error, before decoding = 0, after decoding = 0
Error = [0 1 0 0 0 0 0 0 0 0 0 0 0 0], # Qbits in error, before decoding = 0, after decoding = 0
Error = [0 0 1 0 0 0 0 0 0 0 0 0 0 0], # Qbits in error, before decoding = 0, after decoding = 0
Error = [0 0 0 1 0 0 0 0 0 0 0

In [ ]:
# verify for all single bit errors
for errType in range(3):
    for errIndex in range(n):

        # generate channel error
        errCheckRow = np.zeros(n, dtype = 'int')
        errCheckRow[errIndex] = 1
        if(errType == 0):
            errCheckRow = np.append(errCheckRow, errCheckRow) # Y error
        elif(errType == 1):
            errCheckRow = np.append(errCheckRow, np.zeros(n, dtype = 'int')) # X error
        else:
            errCheckRow = np.append(np.zeros(n, dtype = 'int'), errCheckRow) # Z error

        # apply channel error
        channel_error = getGenerator(errCheckRow)
        rx_erry = np.dot(channel_error, tx_encoded) # received qbits with error

        # get syndrome
        syndr = np.zeros([n-k, 1]) # syndrome
        for i in range(n-k):
            syndr[i] = np.dot(rx_erry.transpose(), np.dot(gi[i,:,:], rx_erry))

        # error correction
        recov = SyndromeLookUp(syndr) # error recovery
        rx_encoded = np.matmul(recov.transpose(), rx_erry) # received qbits without error but still encoded
        
        # setup optimizer to decode completely
        P = np.matmul(Gmatrix.transpose(), Gmatrix)
        q = -np.matmul(rx_encoded.transpose(), Gmatrix).flatten()
        x = cp.Variable(rx_encoded.shape[0])
        
        # solve for qubits numerically by minimizing distance
        prob = cp.Problem(cp.Minimize((1/2)*cp.quad_form(x, P) + q.T@x), [A@x == np.zeros(x.shape[0])])
        prob.solve()
        rx_decoded = (NormalizeState(x.value)) # received decoded qbits
        
        print(errCheckRow)

        # print qubit errors
#         print('Error = ' + str(errCheckRow)  + ', # Qbits in error, before decoding = ' + str(np.sum(np.abs(rx_encoded[:,0] - tx_encoded[:,0]) > 1e-5)) + ', after decoding = ' + str(np.sum(np.abs(rx_decoded - tx_qbits.flatten()) > 1e-5 )) )

[1 0 0 0 0 0 0 1 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0 1 0 0 0 0 0]
[0 0 1 0 0 0 0 0 0 1 0 0 0 0]
[0 0 0 1 0 0 0 0 0 0 1 0 0 0]
[0 0 0 0 1 0 0 0 0 0 0 1 0 0]
[0 0 0 0 0 1 0 0 0 0 0 0 1 0]
[0 0 0 0 0 0 1 0 0 0 0 0 0 1]
[1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 1 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 1 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 1 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 1 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 1 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 1 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 1 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 1 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 1 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 1 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 1]


In [ ]:
def analyzeChannel(errCheckRow):
    # apply channel error
    channel_error = getGenerator(errCheckRow)
    rx_erry = np.dot(channel_error, tx_encoded) # received qbits with error

    # get syndrome
    syndr = np.zeros([n-k, 1]) # syndrome
    for i in range(n-k):
        syndr[i] = np.dot(rx_erry.transpose(), np.dot(gi[i,:,:], rx_erry))

    # error correction
    recov = SyndromeLookUp(syndr) # error recovery
    rx_encoded = np.matmul(recov.transpose(), rx_erry) # received qbits without error but still encoded

    # setup optimizer to decode completely
    P = np.matmul(Gmatrix.transpose(), Gmatrix)
    q = -np.matmul(rx_encoded.transpose(), Gmatrix).flatten()
    x = cp.Variable(rx_encoded.shape[0])

    # solve for qubits numerically by minimizing distance
    prob = cp.Problem(cp.Minimize((1/2)*cp.quad_form(x, P) + q.T@x), [A@x == np.zeros(x.shape[0])])
    prob.solve()
    rx_decoded = (NormalizeState(x.value)) # received decoded qbits

    print(errCheckRow)

#     # print qubit errors
#     print('Error = ' + str(errCheckRow)  + ', # Qbits in error, before decoding = ' + str(np.sum(np.abs(rx_encoded[:,0] - tx_encoded[:,0]) > 1e-5)) + ', after decoding = ' + str(np.sum(np.abs(rx_decoded - tx_qbits.flatten()) > 1e-5 )) )

In [ ]:
myIndex1 = 1
while(myIndex1 < 2**(n)):
    # generate weight 1 vectors
    checkRow1 = list(string_format.format("{:b}".format(myIndex1)))
    checkRow1 = np.asarray(list(map(int, checkRow1)))
    
    myIndex2 = myIndex1*2
    while(myIndex2 < 2**n):
        # generate another weight 1 vector
        checkRow2 = list(string_format.format("{:b}".format(myIndex2)))
        checkRow2 = np.asarray(list(map(int, checkRow2)))
        
        #generate weight 2 vector
        checkRow3 = list(string_format.format("{:b}".format(myIndex2)))
        checkRow3 = np.asarray(list(map(int, checkRow3)))
        checkRow3[checkRow1 == 1] = 1
        
        # add weight 2 errors with XX, XY, XZ, ...
        analyzeChannel(np.append(checkRow3, np.zeros(n, dtype = 'int')))
        analyzeChannel(np.append(checkRow3, checkRow1))
        analyzeChannel(np.append(checkRow2, checkRow1))
        
        analyzeChannel(np.append(checkRow3, checkRow2))
        analyzeChannel(np.append(checkRow3, checkRow3))
        analyzeChannel(np.append(checkRow2, checkRow3))
        
        analyzeChannel(np.append(checkRow1, checkRow2))
        analyzeChannel(np.append(checkRow1, checkRow3))      
        analyzeChannel(np.append(np.zeros(n, dtype = 'int'), checkRow3))
               
        myIndex2 = myIndex2*2
        
    myIndex1 = myIndex1*2   

[0 0 0 0 0 1 1 0 0 0 0 0 0 0]
[0 0 0 0 0 1 1 0 0 0 0 0 0 1]
[0 0 0 0 0 1 0 0 0 0 0 0 0 1]
[0 0 0 0 0 1 1 0 0 0 0 0 1 0]
[0 0 0 0 0 1 1 0 0 0 0 0 1 1]
[0 0 0 0 0 1 0 0 0 0 0 0 1 1]
[0 0 0 0 0 0 1 0 0 0 0 0 1 0]
[0 0 0 0 0 0 1 0 0 0 0 0 1 1]
[0 0 0 0 0 0 0 0 0 0 0 0 1 1]
[0 0 0 0 1 0 1 0 0 0 0 0 0 0]
[0 0 0 0 1 0 1 0 0 0 0 0 0 1]
[0 0 0 0 1 0 0 0 0 0 0 0 0 1]
[0 0 0 0 1 0 1 0 0 0 0 1 0 0]
[0 0 0 0 1 0 1 0 0 0 0 1 0 1]
[0 0 0 0 1 0 0 0 0 0 0 1 0 1]
[0 0 0 0 0 0 1 0 0 0 0 1 0 0]
[0 0 0 0 0 0 1 0 0 0 0 1 0 1]
[0 0 0 0 0 0 0 0 0 0 0 1 0 1]
[0 0 0 1 0 0 1 0 0 0 0 0 0 0]
[0 0 0 1 0 0 1 0 0 0 0 0 0 1]
[0 0 0 1 0 0 0 0 0 0 0 0 0 1]
[0 0 0 1 0 0 1 0 0 0 1 0 0 0]
[0 0 0 1 0 0 1 0 0 0 1 0 0 1]
[0 0 0 1 0 0 0 0 0 0 1 0 0 1]
[0 0 0 0 0 0 1 0 0 0 1 0 0 0]
[0 0 0 0 0 0 1 0 0 0 1 0 0 1]
[0 0 0 0 0 0 0 0 0 0 1 0 0 1]
[0 0 1 0 0 0 1 0 0 0 0 0 0 0]
[0 0 1 0 0 0 1 0 0 0 0 0 0 1]
[0 0 1 0 0 0 0 0 0 0 0 0 0 1]
[0 0 1 0 0 0 1 0 0 1 0 0 0 0]
[0 0 1 0 0 0 1 0 0 1 0 0 0 1]
[0 0 1 0 0 0 0 0 0 1 0 0 0 1]
[0 0 0 0 0

### TODO
- Complete generalization/randomization
- Maybe start generating data

In [ ]:
# tx_qbits.flatten()

In [ ]:
# rx_decoded.flatten()

## Rough

In [ ]:
zeroCols = np.zeros(Gmatrix.shape[1])
for i in range(Gmatrix.shape[1]):
    zeroCols[i] = all(Gmatrix[:,i] == np.zeros(Gmatrix.shape[0]))

In [ ]:
np.sum(zeroCols!=1)

16

In [ ]:
np.sum(zeroCols)

112.0

In [ ]:
Gmatrix.shape[1]

128